In [ ]:
!pip install wandb -qqq

In [ ]:
import wandb
api = wandb.Api()

# Change oreilly-class/cifar to <entity/project-name>
runs = api.runs("capi-bgu/capi")
summary_list = [] 
config_list = [] 
name_list = [] 
for run in runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items() if not k.startswith('_')}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

import pandas as pd 
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
all_df = pd.concat([name_df, config_df,summary_df], axis=1)

all_df.to_csv("project.csv")

In [ ]:
test_acc = pd.concat([summary_df[["test accuracy", "test balanced accuracy"]], name_df], axis=1)
test_acc = test_acc.dropna()
test_acc["is_pos"] = test_acc["name"].str.contains(".*pos.*", regex=True)
test_acc["label type"] = test_acc["is_pos"].map({
    True: "positive/negative",
    False: "categorical"
})
for i in range(10):
  test_acc["label type"][i] = "general"

In [ ]:
import seaborn as sns
sns.set(rc={"figure.figsize":(20, 10)}) #width=3, #height=4

sns.scatterplot(data=test_acc, x="test balanced accuracy", y="test accuracy", hue="label type")

In [ ]:
import matplotlib.pyplot as plt

test_err = pd.concat([summary_df[["test mean absolute error", "test mean squared error"]], name_df], axis=1)
test_err = test_err.dropna()
test_err_ensemble = test_err[test_err["name"].str.contains(".*ensemble.*", regex=True)]

sns.scatterplot(data=test_err, x="test mean squared error", y="test mean absolute error")
sns.scatterplot(data=test_err_ensemble, x="test mean squared error", y="test mean absolute error")
plt.legend(["all models", "ensemble models"])

In [ ]:
import scipy.stats as sts

test_mae = test_err["test mean absolute error"].values
test_mae_ensemble = test_err_ensemble["test mean absolute error"].values
print(sts.norm.interval(0.95, loc=test_mae.mean(), scale=test_mae.std()))
print(sts.norm.interval(0.95, loc=test_mae_ensemble.mean(), scale=test_mae_ensemble.std()))

In [ ]:
test_cm = pd.concat([summary_df[["ensemble confusion matrix_table", "testing confusion matrix_table"]], name_df], axis=1)
test_cm["testing confusion matrix_table"][50]